In [1]:
import numpy as np
import pymysql
import configparser
import json
import time

In [2]:
ip = "139.199.102.41"
config = configparser.RawConfigParser()
config.readfp(open('../settings.cfg'))

dbname = config.get('jx3bla', 'username')
dbpwd = config.get('jx3bla', 'password')
db = pymysql.connect(ip,dbname,dbpwd,"jx3bla",port=3306,charset='utf8')
cursor = db.cursor()

d:\develop\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
edition = "3.5.0"
mapdetail = "25人英雄范阳夜变"

sql = """SELECT * FROM XiangZhiStat WHERE edition='3.5.0' AND mapdetail='25人英雄范阳夜变'"""
cursor.execute(sql)
result = cursor.fetchall()

In [3]:
edition = "5.1.0"
mapdetail = "25人普通达摩洞"

sql = """SELECT * FROM XiangZhiStat WHERE edition>='%s' AND mapdetail='%s'"""%(edition, mapdetail)
cursor.execute(sql)
result = cursor.fetchall()

In [4]:
print(result)

(('山雨欲来', '汉承秦制', 0, '2020-10-20', '25人普通达摩洞', '5.1.0', '001b6b6429ea4b2d231ab9e50c00d32b', "{'numeffheal': 783619, 'hardNPC': '未知', 'npcRank': 0, 'maxDpsRank': 1, 'sumDeath': 1, 'healTable': [['余晖', 1389, 0.07638481147131272, 4, 4, 3616, 50, 388, 7.731958762886597], ['余晖', 654, 0.04637213755542822, 4, 4, 4567, 51, 374, 8.181818181818182]], 'printTable': [], 'rateTable': [['余晖', 0.9323931021170234], ['余晖', 0.9029216619398223]], 'npcNum': 0, 'sumHit': 0, 'maxDpsTable': [['27434', 127189.51228061643, '汉承秦制', '22h'], ['36038', 99159.99217216476, '倾慕成仙', '24'], ['40267', 96366.08740989809, '掠城', '21d'], ['20886', 95976.95141969135, '冰棍棍', '6d'], ['7788', 95331.41921151117, '英雄达拉崩吧', '8'], ['28522', 95296.03767636571, '全网最红焚影', '10d'], ['27275', 94878.3484478602, '何故遇', '21d'], ['27353', 94618.82982907361, '汜水', '9'], ['17016', 93725.03508769262, '沧海间', '2d'], ['23770', 88631.01499089481, '少林俗家衍员', '1d'], ['27435', 88485.82080241267, '伞剑仙', '24'], ['20836', 88191.30663995404, '龙剑逍遥', '4m'],

In [5]:
allstat = {}
allstat["score"] = []
for line in result:
    stat = json.loads(line[7].replace("'", '"'))
    for key in stat:
        if key not in allstat:
            allstat[key] = [stat[key]]
        else:
            allstat[key].append(stat[key])
    allstat["score"].append(line[2])

In [7]:
print(allstat)

{'score': []}


In [29]:
for ss in ["npcNum", "npcRank", "numpurge", "npcHealRate", "npcHeal"]:

    tmp = []
    for line in allstat[ss]:
        if type(line) != type(0):
            tmp.append(0)
        else:
            tmp.append(line)
    allstat[ss] = tmp

In [30]:
avgstat = {}
medstat = {}
maxstat = {}
minstat = {}
numstat = {}

for key in allstat:
    data = allstat[key]
    if type(data[0]) == type(0) or type(data[0]) == type(0.0):
        nData = np.array(data)
        print(data, key)
        avgstat[key] = np.mean(nData)
        medstat[key] = np.median(nData)
        maxstat[key] = np.max(nData)
        minstat[key] = np.min(nData)
        numstat[key] = nData.shape[0]
    elif key in ["healTable", "dpsTable", "spareRateList", "rateTable"]:
        bossData = {}
        for record in data:
            for line in record:
                if line[0] not in bossData:
                    bossData[line[0]] = [line[1:]]
                else:
                    bossData[line[0]].append(line[1:])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in bossData:
            nData = np.array(bossData[line])
            avgstat[key].append([line] + list(np.mean(nData, axis=0)))
            medstat[key].append([line] + list(np.median(nData, axis=0)))
            maxstat[key].append([line] + list(np.max(nData, axis=0)))
            minstat[key].append([line] + list(np.min(nData, axis=0)))
            numstat[key].append(nData.shape[0])
    elif key in ["breakList", "rateList"]:
        if key == "breakList":
            subkey = "bossBreakDict"
        elif key == "rateList":
            subkey = "bossRateDict"
        occData = {}
        for i in range(len(data)):
            for line in data[i]:
                if line[3] not in occData:
                    occData[line[3]] = [[line[1]] + allstat[subkey][i][line[0]]]
                else:
                    occData[line[3]].append([line[1]] + allstat[subkey][i][line[0]])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        avgstat[subkey] = {}
        medstat[subkey] = {}
        maxstat[subkey] = {}
        minstat[subkey] = {}
        numstat[subkey] = {}
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append([line, np.mean(nData[:, 0]), "平均数", line])
            medstat[key].append([line, np.median(nData[:, 0]), "中位数", line])
            maxstat[key].append([line, np.max(nData[:, 0]), "最大值", line])
            minstat[key].append([line, np.min(nData[:, 0]), "最小值", line])
            numstat[key].append(nData.shape[0])
            avgstat[subkey][line] = list(np.mean(nData[:, 1:], axis=0))
            medstat[subkey][line] = list(np.median(nData[:, 1:], axis=0))
            maxstat[subkey][line] = list(np.max(nData[:, 1:], axis=0))
            minstat[subkey][line] = list(np.min(nData[:, 1:], axis=0))
            numstat[subkey][line] = nData.shape[0]
            
    elif key in ["npcHealList"]:
        occData = {}
        for i in range(len(data)):
            for line in data[i]:
                if len(line) == 0:
                    continue
                for j in range(0, 1):
                    if line[j][3] not in occData:
                        occData[line[j][3]] = [line[j][1]]
                    else:
                        occData[line[j][3]].append(line[j][1])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append([line, np.mean(nData), "平均数", line])
            medstat[key].append([line, np.median(nData), "中位数", line])
            maxstat[key].append([line, np.max(nData), "最大值", line])
            minstat[key].append([line, np.min(nData), "最小值", line])
            numstat[key].append(nData.shape[0])
    elif key in ["healList"]:
        occData = {}
        for i in range(len(data)):
            for line in data[i]:
                if len(line) != 8:
                    continue
                if line[1] not in occData:
                    occData[line[1]] = [line[2:]]
                else:
                    occData[line[1]].append(line[2:])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append(["平均数", line] + list(np.mean(nData, axis = 0)))
            medstat[key].append(["中位数", line] + list(np.median(nData, axis = 0)))
            maxstat[key].append(["最大值", line] + list(np.max(nData, axis = 0)))
            minstat[key].append(["最小值", line] + list(np.min(nData, axis = 0)))
            numstat[key].append(nData.shape[0])
    elif key in ["hitCount"]:
        hitData = {}
        for line in data:
            for skillkey in line:
                if skillkey not in hitData:
                    hitData[skillkey] = [line[skillkey]]
                else:
                    hitData[skillkey].append(line[skillkey])
        avgstat[key] = {}
        medstat[key] = {}
        maxstat[key] = {}
        minstat[key] = {}
        numstat[key] = {}
        for skillkey in hitData:
            nData = np.array(hitData[skillkey])
            avgstat[key][skillkey] = np.mean(nData)
            medstat[key][skillkey] = np.median(nData)
            maxstat[key][skillkey] = np.max(nData)
            minstat[key][skillkey] = np.min(nData)
            numstat[key][skillkey] = nData.shape[0]
    elif key in ["hardBOSS", "hardNPC", "maxSingleBreakName", "maxSingleRateName"]:
        avgstat[key] = data[0]
        medstat[key] = data[0]
        maxstat[key] = data[0]
        minstat[key] = data[0]
        numstat[key] = data[0]
    elif key in ["maxDpsTable"]:
        occData = {}
        for i in range(len(data)):
            if allstat["maxDpsName"][i] not in ["厌夜", "藤原武裔", "余晖"]:
                continue
            avgstat["maxDpsName"] = allstat["maxDpsName"][i]
            medstat["maxDpsName"] = allstat["maxDpsName"][i]
            maxstat["maxDpsName"] = allstat["maxDpsName"][i]
            minstat["maxDpsName"] = allstat["maxDpsName"][i]
            numstat["maxDpsName"] = allstat["maxDpsName"][i]
            for line in data[i]:
                if line[3] not in occData:
                    occData[line[3]] = [line[1]]
                else:
                    occData[line[3]].append(line[1])
        avgstat[key] = []
        medstat[key] = []
        maxstat[key] = []
        minstat[key] = []
        numstat[key] = []
        for line in occData:
            nData = np.array(occData[line])
            avgstat[key].append([line, np.mean(nData), "平均数", line])
            medstat[key].append([line, np.median(nData), "中位数", line])
            maxstat[key].append([line, np.max(nData), "最大值", line])
            minstat[key].append([line, np.min(nData), "最小值", line])
            numstat[key].append(nData.shape[0])
    elif key in ["bossBreakDict", "bossRateDict", "printTable", "maxDpsName"]:
        pass
    else:
        pass
        print(key)
        print(data[0])
avgstat["printTable"] = []
medstat["printTable"] = []
maxstat["printTable"] = []
minstat["printTable"] = []
numstat["printTable"] = []
print(avgstat)

for key in ["maxDpsTable", "rateList", "breakList"]:
    avgstat[key].sort(key=lambda x:-x[1])
    medstat[key].sort(key=lambda x:-x[1])
    maxstat[key].sort(key=lambda x:-x[1])
    minstat[key].sort(key=lambda x:-x[1])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0.0, 0, 0, 0] score
[783619, 4844944, 4229279, 3483118, 18572966, 3692167, 8336920, 25144810, 17083723, 4052585, 10543984, 935125, 528649, 10392323, 3034158, 8906077, 490129, 2441029, 3152162, 6575596, 6575596, 14608714, 1451306, 6296504, 508743, 2377991, 2560723, 18371748, 2545800, 5000022, 12108779, 1662373, 6755141, 6924316, 9740153, 8059459, 7941171, 916385, 1524290, 12622776, 1140109, 248332, 6414282, 235973, 11945933, 15491416, 9772014, 6790913, 12799108, 0, 14816404, 296858, 5911223, 5939670, 10816521, 3379466, 11560084, 1714889, 244637, 12140917, 5281228, 13809543, 3588480, 490129, 2441029, 538982, 10392323, 6296504, 527811, 9559844, 11462574, 1470133, 538982, 237103, 1692536, 192810, 1880352, 2413

In [34]:
if mapdetail == "25人英雄范阳夜变":
    mapabbr = "yd"
elif mapdetail == "25人英雄敖龙岛":
    mapabbr = "ald"
elif mapdetail == "25人普通达摩洞":
    mapabbr = "ptdmd"

avgInfo = {"server": "统计数据", "id": "平均数", "score": avgstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%saverage"%mapabbr, "statistics": avgstat}
medInfo = {"server": "统计数据", "id": "中位数", "score": medstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%smedian"%mapabbr, "statistics": medstat}
maxInfo = {"server": "统计数据", "id": "最大值", "score": maxstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%smaximum"%mapabbr, "statistics": maxstat}
minInfo = {"server": "统计数据", "id": "最小值", "score": minstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%sminimum"%mapabbr, "statistics": minstat}
numInfo = {"server": "统计数据", "id": "样本数", "score": numstat["score"], "battledate": time.strftime("%Y-%m-%d", time.localtime()), 
           "mapdetail": mapdetail, "edition": edition, "hash": "%snum"%mapabbr, "statistics": numstat}

In [32]:
from painter import XiangZhiPainter

painter = XiangZhiPainter(printint=0)
painter.paint(avgInfo, "average.png")
painter.paint(medInfo, "median.png")
painter.paint(maxInfo, "maximun.png")
painter.paint(minInfo, "minimum.png")

ModuleNotFoundError: No module named 'painter'

In [35]:
import json
import urllib.request

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, time):
            return obj.__str__()
        else:
            return super(NpEncoder, self).default(obj)

def upLoadData(info):
    Jdata = json.dumps(info, cls=MyEncoder)
    jpost = {'jdata': Jdata}
    jparse = urllib.parse.urlencode(jpost).encode('utf-8')
    resp = urllib.request.urlopen('http://139.199.102.41:8009/uploadXiangZhiData', data = jparse)
    res = json.load(resp)
    return res

res = upLoadData(avgInfo)
print(res)
res = upLoadData(medInfo)
print(res)
res = upLoadData(maxInfo)
print(res)
res = upLoadData(minInfo)
print(res)

{'num': 92, 'numOver': 0, 'result': 'success'}
{'num': 93, 'numOver': 0, 'result': 'success'}
{'num': 94, 'numOver': 0, 'result': 'success'}
{'num': 95, 'numOver': 0, 'result': 'success'}


In [31]:
print(maxInfo)

{'server': '统计数据', 'id': '最大值', 'score': 100, 'battledate': '2020-09-23', 'mapdetail': '25人英雄范阳夜变', 'edition': '3.5.0', 'hash': 'ydmaximum', 'statistics': {'score': 100, 'rateTable': [['周贽', 0.9628638537866396], ['厌夜', 0.9680685766492992], ['迟驻', 0.7644777520731318], ['白某', 0.8502291693492818], ['安小逢', 0.857025796131175], ['狼牙精锐', 0.8794934934798339]], 'maxDpsName': '厌夜', 'maxDpsTable': [['22', 153368.69649775108, '最大值', '22'], ['21', 134073.78747454355, '最大值', '21'], ['8', 122725.8688278695, '最大值', '8'], ['7', 120012.79064802699, '最大值', '7'], ['9', 119355.65363094829, '最大值', '9'], ['25', 113793.04531264471, '最大值', '25'], ['2', 112930.01299093067, '最大值', '2'], ['3', 109816.49000363576, '最大值', '3'], ['10', 109172.29234684742, '最大值', '10'], ['24', 106595.41989201418, '最大值', '24'], ['4', 102895.91381802401, '最大值', '4'], ['23', 102798.04419127644, '最大值', '23'], ['6', 102325.48761492998, '最大值', '6'], ['1', 91967.31240236801, '最大值', '1'], ['5', 87099.30871145906, '最大值', '5']], 'numheal': 775

In [32]:
maxInfo["score"] = int(maxInfo["score"])
minInfo["score"] = int(minInfo["score"])
maxInfo["statistics"]["score"] = int(maxInfo["score"])
minInfo["statistics"]["score"] = int(minInfo["score"])
testInfo = {}
stat = maxInfo["statistics"]
for key in stat:
    print(key)
    testInfo[key] = stat[key]
    Jdata = json.dumps(testInfo)

score
rateTable
maxDpsName
maxDpsTable
numheal
maxDpsRank
maxEqualDPS
hardBOSS
npcHealList


TypeError: Object of type int32 is not JSON serializable

In [24]:
print(numInfo)

{'server': '统计数据', 'id': '样本数', 'score': 51, 'battledate': '2020-11-10', 'mapdetail': '25人普通达摩洞', 'edition': '5.1.0', 'hash': 'ptdmdnum', 'statistics': {'score': 51, 'numeffheal': 51, 'hardNPC': '未知', 'npcRank': 51, 'maxDpsRank': 51, 'sumDeath': 51, 'healTable': [27, 24, 21, 28, 22, 21], 'rateTable': [27, 24, 21, 28, 22, 21], 'npcNum': 51, 'sumHit': 51, 'maxDpsName': '余晖', 'maxDpsTable': [24, 24, 30, 47, 29, 27, 16, 21, 19, 32, 34, 29, 18, 31, 24, 23, 10, 21, 27, 3, 3, 12, 17, 2, 4, 1], 'overallrate': 51, 'npcHealList': [], 'maxEqualDPS': 51, 'maxDps': 51, 'maxSingleBreakName': '媚骨骨的跟宠', 'numpurge': 51, 'maxSingleRate': 51, 'maxSingleRateName': '亮亮也是酿酿', 'hardBOSS': '未知', 'rateList': [50, 75, 59, 103, 88, 51, 91, 97, 43, 65, 42, 105, 54, 29, 49, 15], 'bossRateDict': {'23': 50, '5': 75, '10': 59, '6': 103, '8': 88, '2': 51, '4': 91, '3': 97, '24': 43, '21': 65, '1': 42, '7': 105, '211': 54, '9': 29, '25': 49, '22': 15}, 'npcHealRate': 51, 'healList': [12, 10, 13, 12], 'numheal': 51, 'np

In [24]:
print(type(maxInfo["statistics"]["numheal"]))



<class 'numpy.int32'>


In [27]:
type(maxInfo["statistics"]["numheal"]) == type(np.array([1])[0])

True